In [1]:
from time import time
from datetime import datetime,timedelta
import requests
import hashlib
import hmac
import base64
from urllib.parse import urlparse
import pandas as pd #dataframes, easier to use

# For SSL use on Mac
import os
import sys
import certifi
os.environ['REQUESTS_CA_BUNDLE'] = os.path.join(os.path.dirname(sys.argv[0]), certifi.where())


## Use Coinbase API to download price data

In [2]:
#Details
#If either one of the start or end fields are not provided then both fields will be ignored. If a custom time range is not declared then one ending now is selected.
#The granularity field must be one of the following values: {60, 300, 900, 3600, 21600, 86400}. Otherwise, your request will be rejected. These values correspond to 
# #timeslices representing one minute, five minutes, fifteen minutes, one hour, six hours, and one day, respectively.
#If data points are readily available, your response may contain as many as 300 candles and some of those candles may precede your declared start value. 
# #The maximum number of data points for a single request is 300 candles. If your selection of start/end time and granularity will result in more than 300 data points,
# your request will be rejected. If you wish to retrieve fine granularity data over a larger time range, you will need to make multiple requests with new start/end ranges.

#youtube https://www.youtube.com/watch?v=lf92JHVNP0g


#url = "https://api.exchange.coinbase.com/products/BTC-USD/candles?granularity=60&start=1645070640&end=1645069740"

#300min, 5 hours
def getBTCPrice(timeEnd):
    granularity = 60 #60 seconds
    delta = timedelta(minutes=granularity/60)
    #max request size is 300 candles
    timeStart = timeEnd - (300*delta)
    timeStart = timeStart.isoformat()
    timeEnd = timeEnd.isoformat() #must be isoformat

    url="https://api.exchange.coinbase.com/products/{product_id}/candles"
    #https://api-public.sandbox.exchange.coinbase.com/products/BTC-USD/book?level=2
    #live is api.exchange.coinbase.com/products/BTC-USD/book?level=1
    product_id = "BTC-USD"

    requestParam  = {
        "start":timeStart,
        "end":timeEnd,
        "granularity":granularity
    }

    url = url.format(product_id=product_id)
    print("URL: ",url)

    headers={
        "Content-Type":"application/json",
        "ACCEPT":"application/json"
    }

    response = requests.get(url,headers=headers, params=requestParam)

    responseJson = response.json()
    df = pd.DataFrame(response.json(),columns=["timestamp","open","high","low","close","volume"])

    #convert timestamp to a date/time
    df["date"] = pd.to_datetime(df["timestamp"], unit='s')
    df = df[["date","open","high","low","close","volume"]]
    df.set_index("date",inplace=True)
    print(str(timeEnd))
    df.to_csv("BC_BTC_prices.2023.03.04.1min.csv", mode="a",header=False)
    #[timestamp, price_low, price_high, price_open, price_close, volume]

In [ ]:
#request from now, then change the date earlier by 5 hours, n number of times

timeEnd = datetime(2015,10,17,4,58,00)
for i in range(0,1968):    #Use a number divisible by 24. 24 gets 5 full days, such as 1920 or 3840.
    getBTCPrice(timeEnd)
    timeEnd = timeEnd - timedelta(hours=5)

## View data

In [3]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [4]:
# Import data
data = pd.read_csv("BC_BTC_prices.2023.03.04.1min.csv", header=None, index_col=0)

# Rename the index column to "Date"
data = data.rename_axis("Date")

# Add headers
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Reverse the order of rows in data frame, so that earlier times come first in the data.
data = data.iloc[::-1]


In [6]:
# View data
dataview = data.tail(1000).copy()
print(dataview)

# Chart price data (last 1000 prices)
fig = go.Figure(data=[go.Candlestick(x=dataview.index, open=dataview['Open'], high=dataview['High'], low=dataview['Low'], close=dataview['Close'])])
fig.update_layout(title='Price', title_x=0.5)
fig.show()

                         Open      High       Low     Close     Volume
Date                                                                  
2023-03-03 17:20:00  22384.37  22398.01  22387.45  22385.93   5.753664
2023-03-03 17:21:00  22385.94  22393.64  22385.94  22391.89   3.673021
2023-03-03 17:22:00  22391.90  22409.56  22391.90  22403.03   5.038206
2023-03-03 17:23:00  22402.83  22408.39  22403.82  22405.04  14.314233
2023-03-03 17:24:00  22393.03  22405.05  22405.01  22394.85  16.891692
...                       ...       ...       ...       ...        ...
2023-03-04 09:55:00  22348.05  22351.93  22350.15  22348.85   1.237495
2023-03-04 09:56:00  22348.75  22351.09  22348.88  22349.08   0.422604
2023-03-04 09:57:00  22348.56  22350.15  22349.08  22349.28   0.445601
2023-03-04 09:58:00  22348.23  22351.41  22349.16  22351.41   0.850206
2023-03-04 09:59:00  22351.40  22353.75  22351.40  22352.49   1.626204

[1000 rows x 5 columns]
